# cuaderno de entrenamiento de `Forward Tacotron`(en español)
Este cuaderno ha sido desarrollado por [rmcpantoja](https://github.com/rmcpantoja)
## créditos:

* repositorio de [as-ideas/ForwardTacotron](https://github.com/as-ideas/ForwardTacotron).


### ¡importante!:

* Este cuaderno todavía está en fase de desarrollo, por lo que es posible que algunas funciones no estén disponibles. Por citar algunas son los entrenamientos de vocoder y el tema de los alineamientos. El cuaderno se actualizará constantemente y eliminaré este aviso en cuanto esté terminado y las pruebas hayan concluido.
* Por ahora, este cuaderno no está óptimo para entrenar con datasets pequeños. Estoy planeando hacer un modelo preentrenado con un dataset 24 horas. Después de ello, ya se podrán entrenar este tipo de datasets.


última actualización: 16/11/2022

In [ ]:
#@markdown ### comprobar la GPU asignada
#@markdown ---
#@markdown Necesitas una t4 como mínimo, ya que el proceso de entrenamiento será en mayor tiempo a diferencia de [tacotron2](http://github.com/nvidia/tacotron2). Si tienes una GPU como k80, vé a la barra de menús y selecciona entorno de ejecución-desconectarse y eliminar entorno de ejecución.
#@markdown * También puedes ejecutar este cuaderno sin GPU (no recomendado) desactivando la aceleración de hardware en la configuración del cuaderno.
#@markdown * Sin embargo, ten en cuenta que el tiempo total requerido para ejecutar los entrenamientos será mucho mayor que en una GPU, y podría tardar hasta semanas en completarse.
!nvidia-smi -L

In [ ]:
#@markdown ### montar Google drive
#@markdown ---
#@markdown Esto es muy importante para almacenar los puntos de control y los conjuntos de datos procesados con los que Forward Tacotron podrá trabajar. Sin embargo, algunas notas importantes:
#@markdown * Es importante que verifiques tu espacio de almacenamiento en [Drive](http://drive.google.com/). De acuerdo al tamaño del dataset, necesitas calcular una mayor cantidad de espacio disponible.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@markdown ## Iniciar el proceso de instalación
#@markdown ---
#@markdown Esto instalará el sintetizador y otras dependencias importantes.
%cd /content
import os
from os.path import exists
if (not os.path.exists("/content/ForwardTacotron")):
  print("clonando repositorio...")
  !git clone https://github.com/as-ideas/ForwardTacotron
else:
  print("El repositorio de trabajo ya existe. Saltando...")
# pip:
!pip install numba librosa pyworld phonemizer==2.2.2 webrtcvad PyYAML dataclasses soundfile scipy tensorboard matplotlib unidecode inflect
#!pip install git+https://github.com/wkentaro/gdown.git
%cd /content/ForwardTacotron
#apt:
!apt install espeak-ng
!wget https://github.com/mikefarah/yq/releases/download/v4.29.2/yq_linux_amd64.tar.gz
!tar -xvf yq_linux_amd64.tar.gz
!mv /content/ForwardTacotron/yq_linux_amd64 /content/ForwardTacotron/yq
#!bash install-man-page.sh
print("Listo")

In [ ]:
#@markdown ### configuración
#@markdown Estas son algunas configuraciones con las que podremos modificar ajustes relacionados a datos y entrenamiento.
#@markdown ---
#@markdown #### nombre deseado para el modelo
tts_model_id = "myTTS" #@param {type:"string"}
!./yq -i '.tts_model_id = "{tts_model_id}"' "config.yaml"
#@markdown ---
#@markdown #### Nombre deseado para el vocoder (si vas a entrenar uno)
voc_model_id = "myvocoder" #@param {type:"string"}
!./yq -i '.voc_model_id = "{voc_model_id}"' "config.yaml"
#@markdown ---
#@markdown #### Por favor, elige el tipo de modelo que se va a entrenar con este conjunto de datos
tts_model = "forward_tacotron" #@param ["forward_tacotron", "fast_pitch"]
!./yq -i '.tts_model = "{tts_model}"' "config.yaml"
#@markdown ---
#@markdown #### directorio de salida
#@markdown (se recomienda guardarlo en drive)
data_path = "/content/drive/MyDrive/ForwardTacotron" #@param {type:"string"}
if not os.path.exists(data_path):
  os.makedirs(data_path)
  os.makedirs(data_path+"/data")
!./yq -i '.data_path = "{data_path}/data/"' "config.yaml"
#@markdown ---
#@markdown #### número de validaciones (no se recomienda cambiar)
n_val = 200 #@param {type:"integer"}
!./yq -i '.preprocessing.n_val = {n_val}' "config.yaml"
#@markdown ---
#@markdown #### Elegir la bariación de idioma en el que tiene este conjunto de datos
#@markdown * tengamos en cuenta que "es" equivale a español de españa (recomendado) y es-419 equivale al español de latinoamérica.
language = 'es' #@param ["es", "es-419"]
!./yq -i '.preprocessing.language = "{language}"' "config.yaml"
#@markdown ---
cleaner_name = 'no_cleaners'
!./yq -i '.preprocessing.cleaner_name = "{cleaner_name}"' "config.yaml"
#@markdown #### Intervalo de pasos para generar señales de entrenamiento del modelo
#@markdown Aquí podremos configurar cada cuántos pasos se generarán figuras, imágenes, visuales y audio, es decir, el progreso del entrenamiento en que se podrá ver en tensorboard (más adelante).
#@markdown * nota: esta configuración aplicará en todos los modelos: Tacotron, Forward_tacotron and FastPitch.
plot_every = 1000 #@param {type:"integer"}
!./yq -i '.tacotron.training.plot_every = {plot_every}' "config.yaml"
!./yq -i '.forward_tacotron.training.plot_every = {plot_every}' "config.yaml"
!./yq -i '.fast_pitch.training.plot_every = {plot_every}' "config.yaml"
#@markdown ---
!./yq -i '.forward_tacotron.training.min_attention_sharpness = 0.25' "config.yaml"
!./yq -i '.forward_tacotron.training.min_attention_alignment = 0.5' "config.yaml"

## trabajando con el conjunto de datos

Puedes saltarte estas celdas si ya preprocesaste un dataset por primera vez y quieres entrenarlo en el último punto de control que se haya guardado. De lo contrario, expande esta sección y lee las instrucciones de cada celda.

In [ ]:
import zipfile
import os
import os.path
#@markdown ### procesamiento del conjuntos de datos
#@markdown ---
#@markdown Esto extraerá el conjunto de datos, ará unos retoques en las transcripciones y finalmente lo proccesa. Se crearán datasets de mel, estos formarán parte para el proceso de entrenamiento.
#@markdown * Nota: si vas a preprocesar conjuntos de datos de mayor tamaño, se recomienda tener más espacio disponible en drive.
#@markdown ---
#@markdown #### ruta de los audios. Ellos deberán almacenarse en un archivo zip
wavs_path = "/content/drive/MyDrive/Wavs_m.zip" #@param {type:"string"}
#@markdown ---
#@markdown #### ruta de transcripción
list_path = "/content/drive/MyDrive/list_m.txt" #@param {type:"string"}
list_filename = os.path.basename(list_path).split('/')[-1]
#@markdown ---
%cd /content/ForwardTacotron
!mkdir /content/ForwardTacotron/wavs
if zipfile.is_zipfile(wavs_path):
  !unzip -j "$wavs_path" -d /content/ForwardTacotron/wavs
else:
  print("Aviso: la ruta de audios no es un archivo comprimido.")

if not os.path.exists(list_path):
  raise Exception("Error: el archivo de transcripción no existe, inténtelo de nuevo por favor.")
else:
  !cp $list_path /content/ForwardTacotron
if list_path.endswith('.txt'):
  print("Arreglando transcripción...")
  !mv /content/ForwardTacotron/$list_filename /content/ForwardTacotron/list.csv
  !sed -i -- 's,.wav|,|,g' "/content/ForwardTacotron/list.csv"
  !sed -i -- 's,wavs/,,g' "/content/ForwardTacotron/list.csv"
print("Ejecutando procesamiento...")
!python preprocess.py --path /content/ForwardTacotron
print("listo")

### ¡Precaución! Debes ejecutar esta celda si tienes un conjunto de datos en tu forward tacotron y quieres entrenar otro. Los contenidos se borrarán.

In [ ]:
#@markdown ### borrar el conjunto de datos actual (si existe)
#@markdown ---
#@markdown Debido a que los datasets se encuentran en la carpeta de trabajo, es posible que necesites entrenar otro datasetp. Si es así, ejecuta esta celda para hacerlo.
# conjunto de datos
!rm -rf /content/ForwardTacotron/*.csv
!rm -rf /content/ForwardTacotron/*.wav
!rm -rf /content/ForwardTacotron/*.zip
# preprocesado:
!rm -rf /content/ForwardTacotron/*.npy
!rm -rf /content/ForwardTacotron/data/*

## ¡A entrenar!
Esta serie de pasos requerirán de tiempo para conseguir un entrenamiento estable y tras horas, y a veces algunos días, obtener los resultados finales. Por favor, sugiero leer atentamente las indicaciones de cada una de las celdas.
### falta por hacer:
* Una opción por si el usuario quiere entrenar un vocoder.
* Hacer un modelo preentrenado español con un dataset de 24 horas para poder entrenar datasets más pequeños. Esto puede durar de hasta 2-3 semanas.
  * Cuando este proceso termine, permitir la descarga de ellos en el apartado del entrenamiento y hacer un cuadro de selección con modelos pre-entrenados.

Estas características se agregarán pronto.

In [ ]:
#@markdown ### aplicar parches
#@markdown ---
#@markdown Antes de continuar, te recomiendo ejecutar esta celda para parchear las funciones de rutas.
#@markdown * Este parche arregla la ruta de guardado de los modelos que se van generando, y los configura para que se guarden en la carpeta {forwardTacotron en tu [drive](http://drive.google.com/).
#@markdown * Saltando la creación de este parche los modelos generados se guardarán en la carpeta raíz del proyecto en lugar de la carpeta ForwardTacotron creada.
%%writefile /content/ForwardTacotron/utils/paths.py
import os
from pathlib import Path


class Paths:
    """Manages and configures the paths used by WaveRNN, Tacotron, and the data."""
    def __init__(self, data_path, voc_id, tts_id):
        self.base = Path(__file__).parent.parent.expanduser().resolve()

        # Data Paths
        self.data = Path(data_path).expanduser().resolve()
        self.quant = self.data/'quant'
        self.mel = self.data/'mel'
        self.gta = self.data/'gta'
        self.alg = self.data/'alg'
        self.raw_pitch = self.data/'raw_pitch'
        self.phon_pitch = self.data/'phon_pitch'
        self.phon_energy = self.data/'phon_energy'

        self.model_output = self.base / 'model_output'

        self.voc_checkpoints = self.data/'../checkpoints'/f'{voc_id}.wavernn'
        self.voc_top_k = self.voc_checkpoints/'top_k_models'
        self.voc_log = self.voc_checkpoints/'logs'

        self.taco_checkpoints = self.data/ '../checkpoints' / f'{tts_id}.tacotron'
        self.taco_log = self.taco_checkpoints / 'logs'

        self.forward_checkpoints = self.data/'../checkpoints'/f'{tts_id}.forward'
        self.forward_log = self.forward_checkpoints/'logs'

        self.create_paths()

    def create_paths(self):
        os.makedirs(self.data, exist_ok=True)
        os.makedirs(self.quant, exist_ok=True)
        os.makedirs(self.mel, exist_ok=True)
        os.makedirs(self.gta, exist_ok=True)
        os.makedirs(self.alg, exist_ok=True)
        os.makedirs(self.raw_pitch, exist_ok=True)
        os.makedirs(self.phon_pitch, exist_ok=True)
        os.makedirs(self.phon_energy, exist_ok=True)
        os.makedirs(self.voc_checkpoints, exist_ok=True)
        os.makedirs(self.voc_top_k, exist_ok=True)
        os.makedirs(self.taco_checkpoints, exist_ok=True)
        os.makedirs(self.forward_checkpoints, exist_ok=True)

    def get_tts_named_weights(self, name):
        """Gets the path for the weights in a named tts checkpoint."""
        return self.taco_checkpoints / f'{name}_weights.pyt'

    def get_tts_named_optim(self, name):
        """Gets the path for the optimizer state in a named tts checkpoint."""
        return self.taco_checkpoints / f'{name}_optim.pyt'

    def get_voc_named_weights(self, name):
        """Gets the path for the weights in a named voc checkpoint."""
        return self.voc_checkpoints/f'{name}_weights.pyt'

    def get_voc_named_optim(self, name):
        """Gets the path for the optimizer state in a named voc checkpoint."""
        return self.voc_checkpoints/f'{name}_optim.pyt'




In [ ]:
#@markdown ### Ejecutar la extensión tensorboard
#@markdown --
#@markdown El tensorboard sirve para visualizar el proceso de entrenamiento del modelo. Ten en cuenta que si quieres visualizar esto, puedes ir a las pestañas **audio**, **image** o **scalars**.
%load_ext tensorboard
%tensorboard --logdir "{data_path}/checkpoints"
import tensorflow as tf
import datetime

In [ ]:
#@markdown ### Entrenamiento 1: Tacotron
#@markdown ---
#@markdown Un punto muy a tomar en cuenta entre Tacotron y Forward tacotron es la división del entrenamiento.
#@markdown * En este caso, se entrenará un modelo tacotron que será dividido en 10k pasos. Es decir, se crearán modelos cada 10k pasos y puede ir aumentando el espacio de almacenamiento. Cada modelo pesa un aproximado de 100 mb. En total son cuatro divisiones, por lo que tendremos un resultado de 40k pasos.
#@markdown * Asimismo, cuando se cumplan 10k pasos, comenzará un nuevo entrenamiento, dividiendo el número de épocas pero multiplicando el número de pasos por época.
#@markdown
#@markdown Sin más, ¡A entrenar!.
!python train_tacotron.py

In [ ]:
#@markdown ### entrenamiento 2: ForwardTacotron
#@markdown ---
#@markdown Esto entrenará el modelo final para forward tacotron, tomando en cuenta el dataset pre-procesado junto a los alineamientos y las condiciones de tono y el modelo tacotron generados en cuales trabajamos en las celdas anteriores.
#@markdown * asimismo, tendremos una división de dos modelos, 150k pasos cada uno.
#@markdown * Ten en cuenta que se tomará en cuenta la atención basándose en el modelo tacotron. Si se está entrenando con pocos archivos debido a la mala atención (podemos darnos cuenta de esto cuando comienza el entrenamiento) hay un problema en el conjunto de datos, así que por favor procura revisarlo, arreglar lo que sea necesario, aumentar más datos o revisar atentamente el Tensorboard.
!python train_forward.py